# Frappe dataset

## Import

In [ ]:
import pandas as pd
import numpy as np
import os.path

## Load dataset

In [ ]:
# open the dataset
left = pd.read_csv('Datasets/frappe dataset/frappe.csv', sep="\t")
right = pd.read_csv('Datasets/frappe dataset/meta.csv', sep="\t")
df = pd.merge(left, right[['item', 'category', 'language']], on=["item"])
df

In [ ]:
print(f'min rating: {df.cnt.min()} \t max rating: {df.cnt.max()} \t mean rating: {df.cnt.mean()}')

## Dataset preprocessing
Context features:
- daytime
- weekday
- isweekend
- weather	

In [ ]:
# delete columns that are not needed
del df['homework']
del df['city']
del df['country']

In [ ]:
# delete rows where in any column there is a 'unknown' value
df = df[~df.eq('unknown').any(1)]
df = df.reset_index(drop=True)

# make user and items id start from 0
df.user = pd.factorize(df.user)[0]
df.item = pd.factorize(df.item)[0]

# make rating binary
df['cnt'] = df['cnt'].apply(lambda x: 1 if x > 4 else 0)

#rename rating column
df = df.rename(columns={'cnt': 'rating'})

# merge game categories under a single label 'Games'
game_categories = ['Brain & Puzzle','Arcade & Action','Cards & Casino','Racing','Sports Games','Brain', 'Casual']
book_categories = ['Books & Reference', 'Comics', 'Libraries & Demo']
df.loc[df['category'].isin(game_categories), 'category'] = 'Games'
df.loc[df['category'].isin(book_categories), 'category'] = 'Books'

# binarize language label: 1 if english, 0 otherwise
df['language'] = df['language'].apply(lambda x: 'english' if x == 'en' else 'other')

### Add user features
For each user find:
- his favourite app category
- if it uses paid apps
- on which day he used the most applications
- with what weather he used more applications

In [ ]:
fav_categories = np.zeros((df.user.nunique(), 2), dtype=object)
for user in df.user.unique():
    # group by category and sum ratings
    grouped = df[['user', 'category', 'rating']][df.user == user].groupby(['user', 'category']).sum().sort_values('rating')
    fav_category = grouped.tail(1).index.get_level_values(1).tolist()[0] # get category with highest rating sum
    fav_categories[user,:] = [user, fav_category] # add to numpy array of (user, fav_category)
    
fav_categories = pd.DataFrame(fav_categories, columns=['user', 'fav_category']) # numpy to dataframe
df = pd.merge(df, fav_categories, on=['user'])

In [ ]:
df.cost = df.cost.eq('paid').mul(1) # convert paid/free to 1/0   TODO:  andrebbe messo a zero quando rating == 0
paid_apps = df[['user', 'cost']].groupby(['user'], as_index=False).any() # find if a user runs paid apps
paid_apps = paid_apps.rename(columns={'cost': 'uses_paid_apps'})
df = pd.merge(df, paid_apps, on=['user'])

In [ ]:
fav_days = np.zeros((df.user.nunique(), 2), dtype=object)
for user in df.user.unique():
    # group by day and sum ratings
    grouped = df[['user', 'weekday', 'rating']][df.user == user].groupby(['user', 'weekday']).sum().sort_values('rating')
    fav_day = grouped.tail(1).index.get_level_values(1).tolist()[0] # get day with highest rating sum
    fav_days[user,:] = [user, fav_day] # add to numpy array of (user, fav_day)
    
fav_days = pd.DataFrame(fav_days, columns=['user', 'fav_day']) # numpy to dataframe
df = pd.merge(df, fav_days, on=['user'])

In [ ]:
fav_weathers = np.zeros((df.user.nunique(), 2), dtype=object)
for user in df.user.unique():
    # group by weather and sum ratings
    grouped = df[['user', 'weather', 'rating']][df.user == user].groupby(['user', 'weather']).sum().sort_values('rating')
    fav_weather = grouped.tail(1).index.get_level_values(1).tolist()[0] # get weather with highest rating sum
    fav_weathers[user,:] = [user, fav_weather] # add to numpy array of (user, fav_weather)
    
fav_weathers = pd.DataFrame(fav_weathers, columns=['user', 'fav_weather']) # numpy to dataframe
df = pd.merge(df, fav_weathers, on=['user'])

In [ ]:
df

In [ ]:
one_hot = ['daytime', 'weekday', 'isweekend', 'weather', 'category', 'language', 'cost', 'fav_category', 'fav_day', 'fav_weather', 'uses_paid_apps']

# convert categorical data to one-hot encoding
for col in one_hot:
  df = pd.get_dummies(df, columns=[col], prefix = [col])

df

In [ ]:
df.to_csv('Datasets/frappe dataset/frappe_final.csv', index = False) # save final dataset to CSV

# keep only rating == 1 and drop duplicate to build matrix factorization dataset
df = df[['user', 'item', 'rating']][df.rating == 1]
df = df.drop_duplicates()
# make user and items id start from 0
df.user = pd.factorize(df.user)[0]
df.item = pd.factorize(df.item)[0]
df.to_csv('Datasets/frappe dataset/frappe_matrix_factorization.csv', index = False)